In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

In [ ]:
# reading data
df = pd.read_csv('/content/NaturalRemedy.csv')
df.head()

,Age,Gender,Symptom Duration,Pain Location,Skin Condition,Respiratory Issues,Nausea/Vomiting,Body Temperature,Blood Pressure,Diarrhea,Fatigue,Bleeding/Blood Loss,Wound/Injuries,Consciousness,Dehydration,Heart Rhythm,Treatment,Recommended Use,Unnamed: 18
0,Baby,Male,Short,Stomach,Normal,Normal,Yes,Normal,Normal,Yes,Yes,NaN,NaN,Alert,Moderate,Fast,Mint,Steep fresh mint leaves in boiling wayer for 5...,NaN
1,Child,Female,Medium,Head,Normal,Normal,No,Normal,Low,No,Yes,NaN,NaN,Very Confused,NaN,Slow,Mint,Steep fresh mint leaves in boiling wayer for 5...,NaN
2,Teenager,Male,Long,Body,Inflammation,Coughing,Yes,Hot,High,Yes,Yes,NaN,NaN,Slightly Confused,Moderate,Slow,Mint,Steep fresh mint leaves in boiling wayer for 5...,NaN
3,Adult,Female,Short,Limbs,Swelling,Normal,No,Normal,Normal,No,No,Light,Burns,Alert,NaN,Normal,Cool Water and Dry Cloth,Rinse the burn with cool water for 10-20 minut...,NaN
4,Senior,Male,Medium,Stomach,Normal,Normal,Yes,Normal,Normal,No,No,NaN,NaN,Alert,Moderate,Normal,Sage,Make sage tea,NaN


In [ ]:
# cleaning data
df.columns = df.columns.str.rstrip()
df = df.drop(columns = ['Age', 'Gender', 'Symptom Duration', 'Recommended Use', 'Unnamed: 18'])
df.head()

,Pain Location,Skin Condition,Respiratory Issues,Nausea/Vomiting,Body Temperature,Blood Pressure,Diarrhea,Fatigue,Bleeding/Blood Loss,Wound/Injuries,Consciousness,Dehydration,Heart Rhythm,Treatment
0,Stomach,Normal,Normal,Yes,Normal,Normal,Yes,Yes,NaN,NaN,Alert,Moderate,Fast,Mint
1,Head,Normal,Normal,No,Normal,Low,No,Yes,NaN,NaN,Very Confused,NaN,Slow,Mint
2,Body,Inflammation,Coughing,Yes,Hot,High,Yes,Yes,NaN,NaN,Slightly Confused,Moderate,Slow,Mint
3,Limbs,Swelling,Normal,No,Normal,Normal,No,No,Light,Burns,Alert,NaN,Normal,Cool Water and Dry Cloth
4,Stomach,Normal,Normal,Yes,Normal,Normal,No,No,NaN,NaN,Alert,Moderate,Normal,Sage


In [ ]:
# transforming from categorical to numerical
lbl = {}
for column in df.columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        lbl[column] = le

In [ ]:
# test train
X = df.drop(columns = ['Treatment'])
y = df['Treatment']

In [ ]:
# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# reshaping for model
X_train_rnn = np.expand_dims(X_train, axis = 1)
X_test_rnn = np.expand_dims(X_test, axis = 1)

In [ ]:
# building rnn model
rnn = Sequential()
rnn.add(SimpleRNN(128, input_shape = (X_train_rnn.shape[1], X_train_rnn.shape[2]), return_sequences=False))
rnn.add(Dense(64, activation = 'relu'))
rnn.add(Dense(len(np.unique(y)), activation='softmax'))  # Output layer for classification

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# compiling model
rnn.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
# training model
rnn.fit(X_train_rnn, y_train, epochs = 20, batch_size = 32, validation_split = 0.2)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.9994 - val_accuracy: 0.0000e+00 - val_loss: 2.7659
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 2.8785 - val_accuracy: 0.0000e+00 - val_loss: 2.7528
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0000e+00 - loss: 2.7733 - val_accuracy: 0.2000 - val_loss: 2.7450
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0500 - loss: 2.6810 - val_accuracy: 0.2000 - val_loss: 2.7320
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.2500 - loss: 2.6008 - val_accuracy: 0.2000 - val_loss: 2.7220
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.2500 - loss: 2.5294 - val_accuracy: 0.2000 - val_loss: 2.7081
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4000 - loss: 2.4619 - val_accuracy: 0.2000 - val_loss: 2.6953
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.3500 - loss: 2.4021 - val_accuracy: 0.2000 -

In [ ]:
# receiving user input for treatment recommendation

def get_user_input():
    user_input = {}
    user_input['Pain Location'] = input("Enter Pain Location: ")
    user_input['Skin Condition'] = input("Enter Skin Condition: ")
    user_input['Respiratory Issues'] = input("Enter Respiratory Issues: ")
    user_input['Nausea/Vomiting'] = input("Enter Nausea/Vomiting: ")
    user_input['Body Temperature'] = input("Enter Body Temperature: ")
    user_input['Blood Pressure'] = input("Enter Blood Pressure: ")
    user_input['Diarrhea'] = input("Enter Diarrhea: ")
    user_input['Fatigue'] = input("Enter Fatigue: ")
    user_input['Bleeding/Blood Loss'] = input("Enter Bleeding/Blood Loss: ")
    user_input['Wound/Injuries'] = input("Enter Wound/Injuries: ")
    user_input['Consciousness'] = input("Enter Consciousness: ")
    user_input['Dehydration'] = input("Enter Dehydration: ")
    user_input['Heart Rhythm'] = input("Enter Heart Rhythm: ")

    user_input_encoded = []

    # Encoding user input based on the corresponding label encoders
    for feature, value in user_input.items():
        if feature in lbl:
            encoder = lbl[feature]
            try:
                user_input_encoded.append(encoder.transform([value])[0])
            except ValueError:
                print(f"Error: '{value}' is not a recognized value for {feature}. Please try again.")
                return None  # Return None to indicate an invalid input
        else:
            print(f"Warning: {feature} not found in encoders. Check column names.")

    # Scale and reshape the input for RNN if all inputs are valid
    user_input_rnn_ready = np.expand_dims(user_input, axis=1)

    return user_input_rnn_ready

In [ ]:
# predicting treatment based on model
def predict_best_treatment():
    user_rnn_input = get_user_input()
    if user_rnn_input is None:
        print("Invalid input provided. Please try again.")
        return

    treatment_prediction = rnn.predict(user_rnn_input)
    predicted_treatment_class = np.argmax(treatment_prediction, axis=1)

    # Convert the predicted class back to the original treatment label
    predicted_treatment = lbl['Treatment'].inverse_transform(predicted_treatment_class)

    print(f"The best predicted treatment is: {predicted_treatment[0]}")

In [ ]:
predict_best_treatment()

Enter Pain Location: Stomach
Enter Skin Condition: Normal
Enter Respiratory Issues: Normal
Enter Nausea/Vomiting: Yes
Enter Body Temperature: Normal
Enter Blood Pressure: Normal
Enter Diarrhea: Yes
Enter Fatigue: Yes
Enter Bleeding/Blood Loss: None
Enter Wound/Injuries: None
Enter Consciousness: Alert
Enter Dehydration: None
Enter Heart Rhythm: Fast
Error: 'None' is not a recognized value for Bleeding/Blood Loss. Please try again.
Invalid input provided. Please try again.
